In [85]:

import numpy as np
# import matplotlib.pyplot as plt
# import plotly.express as px
import plotly.graph_objects as go

In [86]:
N = 100000
n_stations = 150

In [87]:
np.random.seed(34)

In [88]:
stations = np.random.uniform(low= 0, high=N, size=(n_stations, 2))


In [89]:
plane_route = np.array([[x,int(0.3 * x)] for x in range(0,int(N/2),1000 )] + [[x, x - int(0.7 * N/2)] for x in range(int(N/2),N+1,1000 )])

In [90]:
def find_closest_point(point, stations):
    d = np.linalg.norm(stations - point, axis=1)
    return np.argmin(d)

In [91]:
def closest_path(route, stations):
    res = [stations[find_closest_point(p, stations), :] for p in route]
    return res

In [92]:
path_long = np.array(closest_path(plane_route, stations))
path = np.unique(path_long, axis=0)


In [93]:
from scipy.spatial.distance import cosine


In [94]:
def compute_penalty(path):
    return sum([cosine(path[i+1]-path[i], path[i]- path[i-1]) for i in range(1,len(path)-1)])



In [95]:
def compute_distances(path, route):
    return np.array([np.linalg.norm(path-r,axis=1)**2 for r in route])
distances = compute_distances(path, plane_route)
distance_dict = {i:min(distances[:, i]) for i in range(len(path))}

In [96]:
filter_indices = [np.argmin(distances[:, i]) for i in range(len(path))]
plane_route_meteo = np.take(plane_route, filter_indices, axis=0)

In [97]:
from collections import namedtuple
import math
Result = namedtuple('Result', ['fpl', 'ogimet'])

class Point(object):
    def __init__(self, value, name=None):
        self.value = value
        if name is None:
            self.name = '{0}_{1}'.format(*value)
        else:
            self.name = name
        self.x = value[0]
        self.y = value[1]
    def xtd_to(self, segment):
        """
        Given the segment AB, computes cross track error
        :param segment: (Point, Point) the segment AB
        :return: float the cross track error
        """
        p1 = segment[0]
        p2 = segment[1]
        x_diff = p2.x - p1.x
        y_diff = p2.y - p1.y
        num = abs(y_diff*self.x - x_diff*self.y + p2.x*p1.y - p2.y*p1.x)
        den = math.sqrt(y_diff**2 + x_diff**2)
        return num / den
    def distance_to(self, other):
        x_diff = other.x - self.x
        y_diff = other.y - self.y
        return  math.sqrt(y_diff**2 + x_diff**2)

def find_strategic(start, end, results):
    """
    Find point you can not suppress without increasing xtd
    :param start: int
    :param end: int
    :param results: [Result]
    :return:
    """
    # search in reverse order to stop at the latest point in the route direction
    # in segment [i, j] we try to remove inner elements by checking the xtd
    length = len(results)
    for k in range(end - 1, start, -1):
        # xtd from ogimet point to fpl segment
        o_xtd = results[k].ogimet.xtd_to(
            (results[k].fpl, results[k + 1].fpl)
        )
        # xtd from fpl point to ogimet segment
        f_xtd = results[k].fpl.xtd_to(
            (results[start].ogimet, results[end].ogimet)
        )
        if abs(f_xtd) > abs(o_xtd):
            fpl_d = -1
            if k < length - 1:
                fpl_d = results[k].fpl.distance_to(results[k+1].fpl)
            if  abs(f_xtd) < fpl_d or fpl_d < 0:
                return k
    return None
def filter_by_xtd(results):
    """
    Here we keep significant ogimet points.
    By significant, I mean points which increase the xtd if missing.
    The algorithm is recursive, if route is A B C D E F
    and ogimet route found is A B'C'D'E'F
    We try to suppress B', if successful we try to suppress C' and so on
    For example if B', C' and E' are not relevant the loop
    will try to suppress B' and C', then it will keep D' and
    start again from D' to suppress E' and keep F
    At the end we try again (recursion) until the route size is constant.
    For information a typical NAT route will reduce from 26 to 15 points
    and a flight to NRT will end with 26 points (starting from 79)
    :param results: [Result]
    :return: [Result]
    """
    res = [results[0]]
    i = -1
    while i < (len(results) - 1):
        i += 1
        j = i + 2
        # we try to remove many consecutive points until it fails
        while j <= len(results) - 1:
            k = find_strategic(i, j, results)
            if k is None:
                j += 1  # no significant point yet, try to extend to next
            else:
                # a significant point was found, store it
                if results[k].ogimet.name not in [o.name for _, o in res]:
                    res.append(results[k])
                i = k - 1  # will start at k on next round
                break
    res.append(results[-1])
    # recursion works, so try it until there is no change
    if len(res) < len(results):
        return filter_by_xtd(res)
    else:
        return res


ogimet_results = [Result(Point(plane_route_meteo[i]), Point(path[i])) for i in range(len(path))]
ogimet_results = filter_by_xtd(ogimet_results)
strategics = [[r.ogimet.x, r.ogimet.y] for r in ogimet_results]
strategics_np = np.array(strategics)
print(len(strategics))

8


In [98]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=stations[:,0], y=stations[:,1],
                    mode='markers',
                    name='stations'))
fig.add_trace(go.Scatter(x=plane_route[:,0], y=plane_route[:,1],
                    mode='lines',
                    name='route'))
fig.add_trace(go.Scatter(x=path[:,0], y=path[:,1],
                    mode='lines',
                    name='meteo'))
fig.add_trace(go.Scatter(x=strategics_np[:,0], y=strategics_np[:,1],
                    mode='lines',
                    name='strategics'))
